In [1]:
using LowLevelFEM
gmsh.initialize()

gmsh.open("cube1.geo")

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000906933s, CPU 0.000895s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000290384s, CPU 0.000258s)
Info    : Meshing 3D

In [2]:
mat = material("cube")
prob = Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 19)


Problem("cube1", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("cube", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 27)

In [3]:
function normalVector(problem, phName)
    gmsh.model.setCurrent(problem.name)
    dim = problem.dim
    non = problem.non
    field = zeros(non * dim)
    ncoord2 = zeros(3 * problem.non)
    numElem = Int[]
    normalvectors = []

    dimTags = gmsh.model.getEntitiesForPhysicalName(phName)
    for idm in 1:length(dimTags)
        dimTag = dimTags[idm]
        edim = dimTag[1]
        etag = dimTag[2]
        if edim != 2
            error("normalVector: physical group is not a surface.")
        end
        elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)
        nodeTags, ncoord, parametricCoord = gmsh.model.mesh.getNodes(edim, etag, true, false)
        ncoord2[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
        ncoord2[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
        ncoord2[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
        for i in 1:length(elemTypes)
            et = elemTypes[i]
            elementName, eldim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
            localNodeCoord2 = zeros(length(localNodeCoord) ÷ 2 * 3)
            localNodeCoord2[1:3:length(localNodeCoord2)] .= localNodeCoord[1:2:length(localNodeCoord)]
            localNodeCoord2[2:3:length(localNodeCoord2)] .= localNodeCoord[2:2:length(localNodeCoord)]
            comp, fun, ori = gmsh.model.mesh.getBasisFunctions(et, localNodeCoord2, "Lagrange")
            h = reshape(fun, :, numNodes)
            comp3, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, localNodeCoord2, "GradLagrange")
            ∇h = reshape(dfun, :, numNodes)
            nnet = zeros(Int, length(elemTags[i]), numNodes)
            nn2 = zeros(Int, dim * numNodes)
            vx = zeros(numNodes)
            vy = zeros(numNodes)
            vz = zeros(numNodes)
            normvec = zeros(3numNodes)
            for j in 1:length(elemTags[i])
                elem = elemTags[i][j]
                for k in 1:numNodes
                    nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
                end
                for k in 1:numNodes
                    v1x = ∇h[1:3:size(∇h,1), k]' * ncoord2[nnet[j, :] * 3 .- 2]
                    v1y = ∇h[1:3:size(∇h,1), k]' * ncoord2[nnet[j, :] * 3 .- 1]
                    v1z = ∇h[1:3:size(∇h,1), k]' * ncoord2[nnet[j, :] * 3 .- 0]
                    v2x = ∇h[2:3:size(∇h,1), k]' * ncoord2[nnet[j, :] * 3 .- 2]
                    v2y = ∇h[2:3:size(∇h,1), k]' * ncoord2[nnet[j, :] * 3 .- 1]
                    v2z = ∇h[2:3:size(∇h,1), k]' * ncoord2[nnet[j, :] * 3 .- 0]
                    vx[k] = v1y * v2z - v1z * v2y
                    vy[k] = v1z * v2x - v1x * v2z
                    vz[k] = v1x * v2y - v1y * v2x
                    v = √(vx[k]^2 + vy[k]^2 + vz[k]^2)
                    vx[k] ./= v
                    vy[k] ./= v
                    vz[k] ./= v
                end
                normvec[1:3:3numNodes] .= vx
                normvec[2:3:3numNodes] .= vy
                normvec[3:3:3numNodes] .= vz
                push!(numElem, elem)
                push!(normalvectors, reshape(normvec,:,1))
                #display("normvec = $normvec")
                #for k in 1:numNodes, l in 1:numNodes
                #    display(∇h[l*dim-(dim-1):l*dim-(dim-dim), k])
                #end
            end
        end
    end
    return LowLevelFEM.VectorField(normalvectors, [;;], [0.0], numElem, 1, :v3D, problem)
end

normalVector (generic function with 1 method)

In [4]:
n = normalVector(prob, "bottom")

MethodError: MethodError: no method matching copyto!(::Float64, ::Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{0}, Tuple{}, typeof(/), Tuple{Float64, Float64}})
The function `copyto!` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  copyto!(!Matched::AbstractArray, ::Base.Broadcast.Broadcasted{<:Base.Broadcast.AbstractArrayStyle{0}})
   @ Base broadcast.jl:928
  copyto!(!Matched::AbstractArray, ::Base.Broadcast.Broadcasted)
   @ Base broadcast.jl:925
  copyto!(!Matched::AbstractArray, ::Any)
   @ Base abstractarray.jl:931
  ...


In [5]:
showElementResults(n, visible=true)

UndefVarError: UndefVarError: `n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [7]:
gmsh.finalize()